### Librerías necesarias

In [ ]:
import pandas as pd
import os
import geopandas as gpd
from geopy.distance import geodesic
import datetime
from collections import OrderedDict

### Funciones para limpieza de datos

In [2]:
def limpiar_nombre(nombre):
    if pd.isna(nombre):
        return ""
    return " ".join(nombre.strip().upper().split())

In [3]:
def encontrar_mejor_match(afc_25):
    resultados = []

    lineas = afc_25['NombreLinea'].dropna().unique()

    # Separar líneas numéricas y no numéricas
    lineas_numericas = [l for l in lineas if str(l)[0].isdigit()]
    lineas_no_numericas = [l for l in lineas if not str(l)[0].isdigit()]

    for linea_a in lineas_no_numericas:
        paradas_a = set(afc_25[afc_25['NombreLinea'] == linea_a]['CodigoParada'].unique())

        mejor_match = None
        max_coincidencias = 0

        for linea_b in lineas_numericas:
            paradas_b = set(afc_25[afc_25['NombreLinea'] == linea_b]['CodigoParada'].unique())
            coincidencias = len(paradas_a & paradas_b)

            if coincidencias > max_coincidencias:
                mejor_match = linea_b
                max_coincidencias = coincidencias

        resultados.append({
            'NombreLinea_NoNumerica': linea_a,
            'MejorMatch_Numerica': mejor_match,
            'CoincidenciasParadas': max_coincidencias
        })

    return pd.DataFrame(resultados)

### Carga y Limpieza de las bases

In [51]:
ruta = "C:/Users/JORGE/OneDrive/Desktop/TIC/Base"
os.makedirs(ruta, exist_ok=True)

In [52]:
#Rutas descargadas de la pagina del gad de Cuenca
rutas_gad = gpd.read_file("C:/Users/JORGE/OneDrive/Desktop/TIC/Rutas_GAD/Red_transporte.shp")
#----------------------------------------------------------------------------------------------------------------------

#Paradas descargadas de la pagina del gad de Cuenca
paradas_gad = gpd.read_file("C:/Users/JORGE/OneDrive/Desktop/TIC/Paradas_GAD/Paradas depuradas.shp")

# Diccionario de correcciones de nombres
reemplazos = {
    "Alimentador 1": "100",
    'Troncal NORTE': '100',
    'YANATURO': 'Molinopamba',
    '3 Claveles': 'L3 Claveles',
    "1B": "1",
    '28 - Sidcay': '28',
    'El Carmen': '9',
    'Turi': '11',
    'Nulti': '34',
    'Carcel': '11B',
    'La Raya': '33',
    'Tejar': '13',
    'Sidcay': '32',
    'Mall del Rio': '13B',
    'Llacao': '31',
    'El verde': '21',
    'Santa Rosa': '30',
    'Rayoloma': '23',
    'Paccha': '29'
}

reemplazos1 = {
    'Florida - Paluncay': '10',
    'EUCALIPTOS': '4',
    'Rumiloma': '7'
}

# Aplicar reemplazos
paradas_gad['LINEA'] = paradas_gad["LINEA"].replace(reemplazos)
paradas_gad['LINEA'] = paradas_gad["LINEA"].replace(reemplazos1)

In [53]:
# Lectura de la base de datos de las validaciones de las smart cards
afc_25 = pd.read_csv("C:/Users/JORGE/OneDrive/Desktop/TIC/Bases/valid_25_oct.csv", dtype={'FechaHoraValidacion' : str})

# Creación de nuevas columnas
afc_25['Hora']= afc_25['FechaHoraValidacion'].str[8:10].astype(int)
afc_25['Minuto']= afc_25['FechaHoraValidacion'].str[10:12].astype(int)
afc_25['Segundo']= afc_25['FechaHoraValidacion'].str[12:].astype(int)
afc_25['Tiempo']= afc_25['Hora']*3600 + afc_25['Minuto']*60 + afc_25['Segundo']

# Diccionario de correcciones de nombres
reemplazos2 = {
    '10 - Florida - Paluncay': '10',
    'Linea 4': '4'
}

# Aplicar reemplazos
afc_25['NombreLinea'] = afc_25["NombreLinea"].replace(reemplazos)
afc_25['NombreLinea'] = afc_25["NombreLinea"].replace(reemplazos2)

len(afc_25) # Número de transacciones realizadas en el día

350581

In [54]:
len(afc_25['CodigoTarjeta'].unique()) # Número de usuarios en el día 

150119

- Eliminar líneas sin información

In [8]:
# Aplicar función de matches
resultados = encontrar_mejor_match(afc_25)

d_resultados = pd.DataFrame(resultados)
d_resultados = d_resultados.sort_values('CoincidenciasParadas', ascending=False)
#----------------------------------------------------------------------------------------------------------------------
#d_resultados.to_excel(os.path.join(ruta, "coincidencias_lineas.xlsx"), index=False)

In [9]:
len(d_resultados)

37

In [55]:
# Diccionario de correcciones de nombres
reemplazos3 = {
    "Molinopamba": "20",
    'Tarqui': '21',
    'El Rocio': '30',
    'Santa Ana': '24',
    "Cruce del Carmen": "9",
    'Baguanchi del Valle': '15',
    'Gualalcay': '24',
    'Chaullayacu': '21'
}

# Aplicar reemplazos
paradas_gad['LINEA'] = paradas_gad["LINEA"].replace(reemplazos3)
afc_25['NombreLinea'] = afc_25["NombreLinea"].replace(reemplazos3)

In [56]:
# Filtrar las líneas de validación con 2 o menos coincidencias
lineas_malas = d_resultados[d_resultados['CoincidenciasParadas'] < 5]['NombreLinea_NoNumerica']

# Contar cuántas validaciones en afc_25 corresponden a esas líneas
validaciones_malas = afc_25[afc_25['NombreLinea'].isin(lineas_malas)]

# AFC_25
afc_25 = afc_25[~afc_25['NombreLinea'].isin(lineas_malas)]

len(validaciones_malas)

6223

In [57]:
len(afc_25)

344358

In [58]:
a = round((1-(344358/350581))*100,3)
a

1.775

- Eliminar validaciones con paradas 'NO DETERMINADA' y líneas 'Sin Inicializar'

In [59]:
# Eliminar espacios en los nombres de las paradas
afc_25['NombreParada'] = afc_25['NombreParada'].apply(limpiar_nombre)
afc_25['CodigoTarjeta'] = afc_25['CodigoTarjeta'].apply(limpiar_nombre)
afc_25['NombreLinea'] = afc_25['NombreLinea'].apply(limpiar_nombre)

len(afc_25[afc_25['CodigoParada']==0])

4741

In [60]:
len(afc_25[afc_25['CodigoLinea']==0])

19

In [61]:
# Eliminar las filas cuyo nombre de parada es 'No Determinado'
afc_25 = afc_25[~afc_25['CodigoParada'].isin([0])]

# Eliminar las filas cuyo Nombre de Linea es 'Sin Inicializar'
afc_25 = afc_25[~afc_25['CodigoLinea'].isin([0])]

len(afc_25)

339603

In [62]:
344358-339603

4755

In [63]:
b = round((4755/350581)*100,3)
b

1.356

- Paradas con inconsistencias

In [64]:
cond = (paradas_gad['ID-PROX PA'] == 3456) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 3406
paradas_gad.loc[cond, 'PARADA2'] = 'ENTRADA A SAN PEDRO'

cond = (paradas_gad['ID-PROX PA'] == 4565) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 673
paradas_gad.loc[cond, 'PARADA2'] = 'DE LOS CHASQUIS'

cond = (paradas_gad['ID-PROX PA'] == 4566) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 512
paradas_gad.loc[cond, 'PARADA2'] = 'ENTRADA PATAMARCA'

cond = (paradas_gad['ID-PARADA'] == 4367) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 2402
paradas_gad.loc[cond, 'PARADA2'] = 'CREA'

cond = (paradas_gad['ID-PARADA'] == 3258) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 3283
paradas_gad.loc[cond, 'PARADA2'] = 'MINIMERCADO EL PLAYON'

cond = (paradas_gad['ID-PARADA'] == 770) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 1299
paradas_gad.loc[cond, 'PARADA2'] = 'MARIANO CUEVA Y AMERICAS'

cond = (paradas_gad['ID-PARADA'] == 4462) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 3411
paradas_gad.loc[cond, 'PARADA2'] = 'SALIDA DE AUQUILULA'

cond = (paradas_gad['ID-PARADA'] == 2995) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 3013
paradas_gad.loc[cond, 'PARADA2'] = 'AV. 24 DE MAYO Y LAS GOLONDRINAS'

cond = (paradas_gad['ID-PARADA'] == 3942) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 3621
paradas_gad.loc[cond, 'PARADA2'] = 'DON ISAAC'

cond = (paradas_gad['ID-PARADA'] == 4438) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 4209
paradas_gad.loc[cond, 'PARADA2'] = 'SALESIANOS'

cond = (paradas_gad['ID-PARADA'] == 526) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 597
paradas_gad.loc[cond, 'PARADA2'] = 'ESCUELA RUILOVA'

cond = (paradas_gad['ID-PARADA'] == 2736) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 2898
paradas_gad.loc[cond, 'PARADA2'] = 'ENTRADA AL DESPACHO'

cond = (paradas_gad['ID-PARADA'] == 51) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 4423
paradas_gad.loc[cond, 'PARADA2'] = 'CANAL DE RIEGO'

cond = (paradas_gad['ID-PARADA'] == 1856) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 1819
paradas_gad.loc[cond, 'PARADA2'] = 'VEGA MUNOZ Y MANUEL VEGA'

cond = (paradas_gad['ID-PARADA'] == 4503) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 4502
paradas_gad.loc[cond, 'PARADA2'] = 'LA Y'

cond = (paradas_gad['ID-PARADA'] == 4437) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 4028
paradas_gad.loc[cond, 'PARADA2'] = 'CAMINO SALADO 2'

cond = (paradas_gad['ID-PARADA'] == 435) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 1211
paradas_gad.loc[cond, 'PARADA2'] = 'VIA LLACAO'

cond = (paradas_gad['ID-PARADA'] == 4505) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 4506
paradas_gad.loc[cond, 'PARADA2'] = 'CENTRO ESPIRITUAL SANTA MARIA'

cond = (paradas_gad['ID-PARADA'] == 4571) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 4534
paradas_gad.loc[cond, 'PARADA2'] = 'PARQUE NARANCAY'

cond = (paradas_gad['ID-PARADA'] == 1429) 
paradas_gad.loc[cond, 'ID-PROX PA'] = 1475
paradas_gad.loc[cond, 'PARADA2'] = 'BUERAN Y CONDOR'

cond = (paradas_gad['ID-PARADA'] == 1190) & (paradas_gad['ID-PROX PA'] == 1190)
paradas_gad.loc[cond, 'ID-PROX PA'] = 1115
paradas_gad.loc[cond, 'PARADA2'] = 'ENTRADA A LA CATOLICA'

# Obtener la geometría de la fila 1158
filtro_fuente = (paradas_gad['ID-PARADA'] == 695)
if filtro_fuente.any():
    # Obtener valores
    nueva_geom = paradas_gad.loc[filtro_fuente, 'geometry'].values[0]
    nueva_lat = paradas_gad.loc[filtro_fuente, 'LATITUD'].values[0]
    nueva_lon = paradas_gad.loc[filtro_fuente, 'LONGITUD'].values[0]

    # Reemplazar en la fila destino
    paradas_gad.loc[paradas_gad['ID-PARADA'] == 4561, 'geometry'] = nueva_geom
    paradas_gad.loc[paradas_gad['ID-PARADA'] == 4561, 'LATITUD'] = nueva_lat
    paradas_gad.loc[paradas_gad['ID-PARADA'] == 4561, 'LONGITUD'] = nueva_lon

# Obtener la geometría de la fila 1158
filtro_fuente = paradas_gad['ID-PARADA'] == 1158
if filtro_fuente.any():
    nueva_geom = paradas_gad.loc[filtro_fuente, 'geometry'].values[0]
    nueva_lat = paradas_gad.loc[filtro_fuente, 'LATITUD'].values[0]
    nueva_lon = paradas_gad.loc[filtro_fuente, 'LONGITUD'].values[0]

    paradas_gad.loc[paradas_gad['ID-PARADA'] == 1076, 'geometry'] = nueva_geom
    paradas_gad.loc[paradas_gad['ID-PARADA'] == 1076, 'LATITUD'] = nueva_lat
    paradas_gad.loc[paradas_gad['ID-PARADA'] == 1076, 'LONGITUD'] = nueva_lon

# Obtener la geometría de la fila 1158
filtro_fuente = paradas_gad['ID-PARADA'] == 425
if filtro_fuente.any():
    nueva_geom = paradas_gad.loc[filtro_fuente, 'geometry'].values[0]
    nueva_lat = paradas_gad.loc[filtro_fuente, 'LATITUD'].values[0]
    nueva_lon = paradas_gad.loc[filtro_fuente, 'LONGITUD'].values[0]

    paradas_gad.loc[paradas_gad['ID-PARADA'] == 388, 'geometry'] = nueva_geom
    paradas_gad.loc[paradas_gad['ID-PARADA'] == 388, 'LATITUD'] = nueva_lat
    paradas_gad.loc[paradas_gad['ID-PARADA'] == 388, 'LONGITUD'] = nueva_lon

- Asignación de longitud y latitud

In [65]:
# Verifica que todas las paradas de df están en paradas_gad
paradas_afc_25 = set(afc_25['CodigoParada'].unique())
paradas_gad_id = set(paradas_gad['ID-PARADA'].unique())

# Encuentra las paradas que están en df pero no en paradas_gad
paradas_faltantes = paradas_afc_25 - paradas_gad_id

# Resultado
if not paradas_faltantes:
    print("Todas las paradas de AFC_25 están presentes en paradas_gad.")
else:
    print(f"Hay {len(paradas_faltantes)} paradas de AFC_25 que no están en paradas_gad:")
    print(paradas_faltantes)

Hay 68 paradas de AFC_25 que no están en paradas_gad:
{np.int64(4608), np.int64(4605), np.int64(4609), np.int64(4610), np.int64(4484), np.int64(4612), np.int64(4613), np.int64(4614), np.int64(4615), np.int64(4616), np.int64(2186), np.int64(4617), np.int64(4618), np.int64(4624), np.int64(4625), np.int64(4627), np.int64(4628), np.int64(4629), np.int64(4633), np.int64(4634), np.int64(2459), np.int64(4635), np.int64(4636), np.int64(4637), np.int64(4638), np.int64(4639), np.int64(4640), np.int64(4641), np.int64(4642), np.int64(4644), np.int64(4645), np.int64(4646), np.int64(4647), np.int64(4648), np.int64(4649), np.int64(4650), np.int64(4396), np.int64(4652), np.int64(2481), np.int64(4407), np.int64(1219), np.int64(3917), np.int64(853), np.int64(4441), np.int64(4057), np.int64(4442), np.int64(4444), np.int64(4448), np.int64(2529), np.int64(4599), np.int64(4589), np.int64(4590), np.int64(4591), np.int64(4592), np.int64(4593), np.int64(4082), np.int64(4594), np.int64(4595), np.int64(1909), np

In [66]:
# Columnas a unir sean del mismo tipo
afc_25['CodigoParada'] = afc_25['CodigoParada'].astype(int)
paradas_gad['ID-PARADA'] = paradas_gad['ID-PARADA'].astype(int)

# Seleccionamos solo las columnas que deseas añadir
columnas_a_agregar = ['SECUENCIA', 'SENTIDO', 'LONGITUD', 'LATITUD', 'ID-PROX PA', 'PARADA2', 'geometry']
paradas_subset = paradas_gad[['ID-PARADA'] + columnas_a_agregar]

# Hacemos el merge
afc_25_completo = afc_25.merge(paradas_subset, how='left', left_on='CodigoParada', right_on='ID-PARADA')

# Opcional: llenar valores faltantes con 0 o vacío según prefieras
afc_25_completo.fillna({'LONGITUD': 0, 'LATITUD': 0, 'ID-PROX PA': '0', 'PARADA2': '0'}, inplace=True)

#len(afc_25_completo[afc_25_completo['LONGITUD']==0])

afc_25_completo['ID-PROX PA'] = (
    afc_25_completo['ID-PROX PA']
    .dropna()
    .astype(float)
    .astype(int)
    #.astype(str)
)

columnas_prox = ['ID-PARADA', 'LATITUD', 'LONGITUD', 'geometry']
paradas_prox = paradas_gad[columnas_prox].copy()

paradas_prox = paradas_prox.rename(columns={
    'ID-PARADA': 'ID-PROX PA',
    'LATITUD': 'LATITUD_PROX',
    'LONGITUD': 'LONGITUD_PROX',
    'geometry': 'geometry_PROX'
})

afc_25_completo = afc_25_completo.merge(
    paradas_prox,
    how='left',
    on='ID-PROX PA'
)

In [67]:
# Eliminar las filas cuya longitud y latitud es 0 
afc_25_completo = afc_25_completo[~afc_25_completo['LONGITUD'].isin([0])]

len(afc_25_completo)

330491

In [68]:
339603-330491

9112

In [69]:
c = round(((9112/350581))*100,3)
c

2.599

- Tiempo estimado entre paradas

In [70]:
afc_25_completo = afc_25_completo.copy()

# Asegurar tipos
afc_25_completo['NombreLinea'] = afc_25_completo['NombreLinea'].astype(str)

# Calcular tramos
resultados_tramos = []

for i, row in afc_25_completo.iterrows():
    origen = (row['CodigoParada'])
    destino = (row['ID-PROX PA'])
    linea = str(row['NombreLinea'])

    lat1, lon1 = row['LATITUD'], row['LONGITUD']
    lat2, lon2 = row['LATITUD_PROX'], row['LONGITUD_PROX']

    # Verifica que ninguna coordenada esté vacía
    if pd.isna(lat1) or pd.isna(lon1) or pd.isna(lat2) or pd.isna(lon2):
        print(f"❌ Coordenadas faltantes para {origen} → {destino}")
        continue

    # Calcular distancia
    p1 = (lat1, lon1)
    p2 = (lat2, lon2)
    distancia = geodesic(p1, p2).meters

    resultados_tramos.append({
        'NombreLinea': linea,
        'CodigoParada': origen,
        'ID-PROX PA': destino,
        'distancia_m': round(distancia, 2)
    })

df_tramos = pd.DataFrame(resultados_tramos)

In [71]:
# Convertir tiempo a minutos
rutas_gad = rutas_gad[rutas_gad['tiempo_rec'].notna()].copy()
rutas_gad['tiempo_rec'] = rutas_gad['tiempo_rec'].astype(str).apply(
    lambda x: x + ':00' if ':' in x and x.count(':') == 1 else x
)
rutas_gad['tiempo_rec'] = pd.to_timedelta(rutas_gad['tiempo_rec'], errors='coerce')
rutas_gad['tiempo_min'] = rutas_gad['tiempo_rec'].dt.total_seconds() / 60

# ------------------------------
# Calcular tiempo y longitud promedio por línea
# ------------------------------

# Limpiar columnas
df_tramos['NombreLinea'] = df_tramos['NombreLinea'].astype(str).str.strip()
rutas_gad['linename'] = rutas_gad['linename'].astype(str).str.strip()

# Filtrar rutas que tienen correspondencia
lineas_validas = rutas_gad['linename'].unique()
df_tramos = df_tramos[df_tramos['NombreLinea'].isin(lineas_validas)].copy()

# Limpiar y convertir longitud
rutas_gad = rutas_gad[rutas_gad['tiempo_min'].notna()]
rutas_gad['length_km'] = rutas_gad['length'].str.replace('km', '').astype(float)

# Agrupar: obtener promedio de tiempo y longitud por línea
rutas_promedio = rutas_gad.groupby('linename', as_index=False).agg({
    'tiempo_min': 'mean',
    'length_km': 'mean'
})

# ------------------------------
# Unir con df_tramos y calcular tiempo estimado
# ------------------------------
df_tramos = df_tramos.merge(
    rutas_promedio,
    left_on='NombreLinea',
    right_on='linename',
    how='left'
)

df_tramos['tiempo_estimado_min'] = df_tramos.apply(
    lambda row: (row['distancia_m'] / (row['length_km'] * 1000)) * row['tiempo_min']
    if pd.notnull(row['length_km']) and row['length_km'] > 0 else None,
    axis=1
)

In [72]:
df_tramos = df_tramos.drop_duplicates(subset=['CodigoParada'])

In [73]:
afc_25_completo = afc_25_completo.merge(
    df_tramos[['CodigoParada', 'ID-PROX PA', 'distancia_m', 'tiempo_estimado_min']],
    on=['CodigoParada'],
    how='left'
)

- Eliminar usuarios con más de 6 transacciones AFC en el día

In [74]:
# Número de usuarios para cada validación
df = afc_25_completo.groupby("CodigoTarjeta").count()
df = df.rename(columns={'CodigoParada': 'Cantidad_validaciones'})

df1 = df.groupby('Cantidad_validaciones').count()

df1 = df1[['NombreParada']].rename(columns={'NombreParada': 'Número_usuarios'})
df1

,Número_usuarios
Cantidad_validaciones,
1,49375
2,52923
3,21303
4,13658
5,4856
6,2539
7,1008
8,599
9,273


In [75]:
# Eliminación de usuarios con más de 6 validaciones 
conteo = afc_25_completo["CodigoTarjeta"].value_counts()
aux = conteo[conteo < 7].index
afc_25_completo = afc_25_completo[afc_25_completo['CodigoTarjeta'].isin(aux)]

len(afc_25_completo)
#len(afc_25_completo['CodigoTarjeta'].unique())

313276

In [76]:
330491-313276

17215

In [77]:
d = round((17215/350581)*100,3)
d

4.91

In [78]:
print(a+b+c+d)

10.64


In [79]:
round((313276/350581)*100,2)

89.36

- Secuencia global

In [80]:
afc_25_completo["Tiempo"] = afc_25_completo.apply(
    lambda row: datetime.time(row["Hora"], row["Minuto"], row["Segundo"]),
    axis=1
)

# Asegúrate de ordenar los datos 
afc_25_completo.sort_values(by=["CodigoLinea", "RegistroMunicipal", "Tiempo"], inplace=True)

# Crear secuencia coherente por línea 
secuencia_global = {}

for linea in afc_25_completo["CodigoLinea"].unique():
    df_linea = afc_25_completo[afc_25_completo["CodigoLinea"] == linea].copy()
    df_linea.sort_values(by=["RegistroMunicipal", "Tiempo"], inplace=True)

    secuencia = []  # Lista de paradas únicas en orden observado

    for _, grupo in df_linea.groupby("RegistroMunicipal"):
        paradas = grupo["CodigoParada"].tolist()
        for i, parada in enumerate(paradas):
            if parada not in secuencia:
                # Insertar entre dos paradas si es posible
                if i > 0 and paradas[i - 1] in secuencia:
                    idx = secuencia.index(paradas[i - 1]) + 1
                    if idx < len(secuencia):
                        if paradas[i] != secuencia[idx]:
                            secuencia.insert(idx, parada)
                    else:
                        secuencia.append(parada)
                else:
                    secuencia.append(parada)

    # Guardar secuencia ordenada para esa línea
    secuencia_global[linea] = OrderedDict((p, i + 1) for i, p in enumerate(secuencia))

# Asignar secuencia a cada validación 
def asignar_secuencia(row):
    linea = row["CodigoLinea"]
    parada = row["CodigoParada"]
    return secuencia_global.get(linea, {}).get(parada, None)

afc_25_completo["SECUENCIA_GLOBAL"] = afc_25_completo.apply(asignar_secuencia, axis=1)

- Tiempo entre transacciones o timbradas

In [81]:
afc_25_completo = afc_25_completo.copy()

# Transformar los datos para el gráfico 
afc_25_completo['TiempoCompleto'] = pd.to_datetime(
    afc_25_completo['Hora'].astype(str).str.zfill(2) + ':' +
    afc_25_completo['Minuto'].astype(str).str.zfill(2) + ':' +
    afc_25_completo['Segundo'].astype(str).str.zfill(2),
    format='%H:%M:%S'
)

# Asegúrate de que esté ordenado correctamente
afc_25_completo = afc_25_completo.sort_values(['CodigoTarjeta', 'TiempoCompleto'])

# Calcular diferencia hacia adelante por tarjeta
afc_25_completo['TiempoEntreTimbradas'] = afc_25_completo.groupby('CodigoTarjeta')['TiempoCompleto'].diff(-1).abs()

# Convertir a minutos
afc_25_completo['TiempoEntreTimbradas_min'] = afc_25_completo['TiempoEntreTimbradas'].dt.total_seconds() / 60

In [82]:
afc_25_completo = afc_25_completo[['CodigoTarjeta', 'Hora', 'Minuto', 'Segundo', 'Tiempo', 'CodigoLinea', 'NombreLinea', 'RegistroMunicipal', 'CodigoParada', 'NombreParada', 'ID-PROX PA_x', 
                    'PARADA2', 'SENTIDO', 'SECUENCIA','SECUENCIA_GLOBAL', 'LATITUD', 'LONGITUD', 'geometry', 'LATITUD_PROX', 'LONGITUD_PROX', 'geometry_PROX', 'TiempoEntreTimbradas_min', 'distancia_m', 'tiempo_estimado_min']]

In [ ]:
#----------------------------------------------------------------------------------------------------------------------
#afc_25_completo.to_excel(os.path.join(ruta, "AFC_25_Completo.xlsx"), index=False)
#afc_25_completo.to_csv(os.path.join(ruta, "afc_25_completo.csv"), index=False)

313276